# CCRB Officer Data 

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import pandas_bokeh
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pandas_bokeh.output_notebook()

from ccrb_utils import *

Loading BokehJS ...

In [3]:
from bokeh.io import show
from bokeh.models import TextInput, ColumnDataSource,ColorBar, DataTable, TableColumn 
from bokeh.models import LinearColorMapper,BasicTicker, HoverTool, Select
from bokeh.plotting import figure


In [8]:
command_list = ["NARCBBX", "WARRSEC",]+ \
               [f"04{i} PCT" for i in range(10)]+ \
               [f"05{k} PCT" for k in [0,2]]   
menu_option = ["Narcotics BX","Warrant Service"]+ \
              [f"4{i} Precinct" for i in range(10)]+ \
              [f"5{k} Precinct" for k in [0,2]]
no_penalty_list = ["no disciplinary action-dup",
                   "no disciplinary action-dismissed",
                   "no disciplinary action-sol", "filed",
                   " retained, without discipline"]
other_list = ["retained, with discipline", " other",
              np.NaN, 0.0]
outcome_list = [np.NaN, "not guilty", 0.0]

# Should add these to the clean up file
url = "ccrb_clean"
ccrb_df = pd.read_csv(url, parse_dates=["Incident.Date"])
ccrb_df = ccrb_df.drop(['Unnamed: 0', 'AsOfDate', 'Unique.Id',
                        'First.Name', 'Last.Name'],
                       axis=1)
ccrb_df.columns = ccrb_df.columns.str.replace(".","_")
ccrb_df.rename(columns={"NYPDDisposition":"NYPD_disposition", 
                        "PenaltyDesc":"Penalty", 
                       "ShieldNo":"Shield_No"}, 
              inplace=True)
ccrb_df.columns = ccrb_df.columns.str.lower()
ccrb_df.dropna(subset=['incident_date'], inplace=True)
ccrb_df.incident_date = pd.to_datetime(ccrb_df.incident_date, format='%Y-%m-%d')
ccrb_df = ccrb_df[ccrb_df['incident_date'] > "1984-01-01"]

ccrb_df.nypd_disposition = ccrb_df.nypd_disposition.apply(lambda x: make_nypd_bins(x,
                                    no_penalty_list,
                                    other_list))
ccrb_df.penalty = ccrb_df.penalty.apply(lambda x: make_penalty_bins(x,
                                       outcome_list))
ccrb_df.penalty = ccrb_df.penalty.apply(lambda x: make_condensed_penalty(x))
ccrb_df.penalty = ccrb_df.penalty.apply(lambda x: fix_instructions(x))
ccrb_df['year'] = ccrb_df.incident_date.dt.year
ccrb_df.shield_no = ccrb_df.shield_no.astype(int)
ccrb_df.full_name = ccrb_df.full_name.str.lower()


In [9]:
ccrb_df[["first", "last"]] = ccrb_df.full_name.str.split(" ",1, expand=True)

In [11]:
ccrb_df.head()

,full_name,rank,command,shield_no,complaint_id,incident_date,fado_type,allegation,board_disposition,nypd_disposition,penalty,year,first,last
2,walter aanonsen,LT,MOUNTED,0,8800504.0,1988-02-15,Force,beat,unsubstantiated,unknown,unknown,1988,walter,aanonsen
3,walter aanonsen,LT,MOUNTED,0,9201176.0,1992-04-02,Abuse of Authority,search - vehicle,unsubstantiated,unknown,unknown,1992,walter,aanonsen
4,walter aanonsen,LT,MOUNTED,0,9201176.0,1992-04-02,Discourtesy,nasty words,unsubstantiated,unknown,unknown,1992,walter,aanonsen
5,walter aanonsen,LT,MOUNTED,0,9201176.0,1992-04-02,Abuse of Authority,detention,unsubstantiated,unknown,unknown,1992,walter,aanonsen
6,walter aanonsen,LT,MOUNTED,0,9600472.0,1995-10-13,Offensive Language,ethnic slur,unfounded,unknown,unknown,1995,walter,aanonsen


In [14]:
test_df = ccrb_df[(ccrb_df['first'].str.startswith("john")) & 
        (ccrb_df['last'].str.startswith("bren"))]

In [16]:
test_df.head()

,full_name,rank,command,shield_no,complaint_id,incident_date,fado_type,allegation,board_disposition,nypd_disposition,penalty,year,first,last
27980,john brennan,LCD,INT UOU,0,9300617.0,1993-03-10,Discourtesy,nasty words,complaint withdrawn,unknown,unknown,1993,john,brennan
27981,john brennan,LCD,INT UOU,0,9300617.0,1993-03-10,Abuse of Authority,other,complaint withdrawn,unknown,unknown,1993,john,brennan
27982,john brennan,LCD,DBSI,0,9500238.0,1995-01-14,Discourtesy,nasty words,unsubstantiated,unknown,unknown,1995,john,brennan
27990,john brennan,SGT,020 PCT,3661,9700745.0,1997-03-01,Abuse of Authority,other,substantiated,unknown,unknown,1997,john,brennan
27991,john brennan,LCD,DBSI,0,9500238.0,1995-01-14,Discourtesy,nasty words,unsubstantiated,unknown,unknown,1995,john,brennan


In [17]:
test_df.groupby(['shield_no']).count()['rank'].index

Int64Index([0, 1564, 3661, 12838], dtype='int64', name='shield_no')

In [81]:
refine_name = test_df.groupby(["full_name", 'shield_no']) \
                    .count()["rank"] \
                    .reset_index() \
                    .full_name \
                    .to_list()
shield_no =test_df.groupby(["full_name", 'shield_no']) \
                    .count()["rank"] \
                    .reset_index() \
                    .shield_no \
                    .to_list()

In [82]:
refine_name

['John Brennan', 'John Brennan', 'John Brennan', 'John Brennan']

In [83]:
name_options = [refine_name[i]+" "+str(shield_no[i]) for i in range(len(refine_name))]

In [84]:
name_options

['John Brennan 0',
 'John Brennan 1564',
 'John Brennan 3661',
 'John Brennan 12838']

In [95]:
test_name = name_options[2].split()
filter_shield = int(test_name[2])

In [96]:
name_filter = test_name[0]+" "+test_name[1]

In [116]:
dt_test = test_df[(test_df.full_name == name_filter)&
        (test_df.shield_no == filter_shield)] \
        [["full_name", "shield_no", "rank", "command"]][:1] \
        .rename(columns={"full_name":"Officer Name",
                        "shield_no": "Shield",
                        "rank": "Rank",
                        "command":"Precinct"})
dt_source = ColumnDataSource(dt_test)
columns = [
        TableColumn(field="Officer Name",),
        TableColumn(field="Shield"),
        TableColumn(field="Rank"),
        TableColumn(field="Precinct")
    ]
data_table = DataTable(source=dt_source, columns=columns, width=400, height=280)


In [117]:
show(data_table)

In [36]:
line_plot = test_df.groupby(['year', 'allegation']).count()["rank"].reset_index()
line_source = ColumnDataSource(line_plot)

e = figure(title = f"Yearly Totals of Complaints",
           x_axis_label = "Year", 
           y_axis_label = "Count", 
           #x_axis_type='datetime', 
           plot_width = 800,
           plot_height = 500,
           toolbar_location = 'above',
           tools='box_zoom, reset')
e.vbar(x='year', top='rank',
       source=line_source,
       line_color="white")
# Hover and Tooltips
tooltips = [('Allegation', '@allegation'),
            ('Count', '@rank'),
           ('Year', '@year')]

e.add_tools(HoverTool(tooltips=tooltips,
                      mode='mouse'))


show(e)

In [37]:
test_df

,full_name,rank,command,shield_no,complaint_id,incident_date,fado_type,allegation,board_disposition,nypd_disposition,penalty,year,first,last
27980,John Brennan,LCD,INT UOU,0,9300617.0,1993-03-10,Discourtesy,nasty words,complaint withdrawn,unknown,unknown,1993,John,Brennan
27981,John Brennan,LCD,INT UOU,0,9300617.0,1993-03-10,Abuse of Authority,other,complaint withdrawn,unknown,unknown,1993,John,Brennan
27982,John Brennan,LCD,DBSI,0,9500238.0,1995-01-14,Discourtesy,nasty words,unsubstantiated,unknown,unknown,1995,John,Brennan
27990,John Brennan,SGT,020 PCT,3661,9700745.0,1997-03-01,Abuse of Authority,other,substantiated,unknown,unknown,1997,John,Brennan
27991,John Brennan,LCD,DBSI,0,9500238.0,1995-01-14,Discourtesy,nasty words,unsubstantiated,unknown,unknown,1995,John,Brennan
28005,John Brennan,LCD,INT UOU,0,9903460.0,1999-07-26,Abuse of Authority,search - frisk,exonerated,unknown,unknown,1999,John,Brennan
28013,John Brennan,LCD,INT UOU,0,200008054.0,2000-11-19,Abuse of Authority,search - vehicle,exonerated,unknown,unknown,2000,John,Brennan
28017,John Brennan,LCD,INT UOU,0,200008054.0,2000-11-19,Abuse of Authority,search - strip,exonerated,unknown,unknown,2000,John,Brennan
28019,John Brennan,LCD,INT UOU,0,200008145.0,2000-11-19,Force,force,exonerated,unknown,unknown,2000,John,Brennan
28021,John Brennan,LCD,INT UOU,0,200102710.0,2001-04-17,Abuse of Authority,property damaged,exonerated,unknown,unknown,2001,John,Brennan


In [134]:
x_tab = pd.crosstab(test_df.board_disposition, test_df.allegation)
data1 = pd.DataFrame(x_tab.stack(), columns=['rate']).reset_index()
heat_source_1 = ColumnDataSource(data1)

In [138]:
np.unique(heat_source_1.data['allegation']).tolist()

['animal',
 'beat',
 'detention',
 'dragged pulled',
 'ethnic slur',
 'nasty words',
 'punch kick',
 'race',
 'search - vehicle']

In [10]:
from bokeh.io import output_file, show
from bokeh.palettes import Spectral11, Turbo256
from bokeh.plotting import figure
from bokeh.transform import factor_cmap, transform

group = test_df.groupby(by=['fado_type']).count()["rank"].sort_values(ascending=False).reset_index()
index_cmap = factor_cmap('fado_type', palette=Spectral11, 
                        factors=group.fado_type.unique(), end=1)

p = figure(plot_width=800, plot_height=300, 
           title="Distribution of Complaint Categories",
           x_range=group.fado_type.unique(),
           toolbar_location=None,
           tooltips=[("FADO", "@fado_type"), ("count", "@rank")])

p.vbar(x='fado_type', top='rank', width=1, source=group,
       line_color="white",fill_color=index_cmap)

p.y_range.start = 0
p.xgrid.grid_line_color = None
p.xaxis.axis_label = "Complaint Category"
p.xaxis.major_label_orientation = 0
p.outline_line_color = None

show(p)

In [11]:
group.fado_type.unique()

array(['Abuse of Authority', 'Force', 'Discourtesy'], dtype=object)

In [12]:
group1 = test_df.groupby(by=['fado_type','allegation']).count()["rank"].sort_values(ascending=False).reset_index()
index_cmap = factor_cmap('fado_type', palette=Spectral11, 
                        factors=group.fado_type.unique(), end=1)

q = figure(plot_width=800, plot_height=500, 
           title="Distribution of Allegations",
           x_range=group1.allegation.unique(),
           toolbar_location=None,
           tooltips=[("allegation", "@allegation"), ("count", "@rank"), ("FADO","@fado_type")])

q.vbar(x='allegation', top='rank', width=1, source=group1,
       line_color="white",fill_color=index_cmap,legend_field='fado_type')

q.y_range.start = 0
q.xgrid.grid_line_color = None
q.xaxis.axis_label = "Allegation"
q.xaxis.major_label_orientation = 1.25
q.outline_line_color = None

show(q)

## Distribution of CCRB board

In [13]:
group = test_df.groupby(by=['board_disposition','allegation']) \
                .count()["rank"] \
                .sort_values(ascending=False) \
                .reset_index()
index_cmap = factor_cmap('board_disposition',
                         palette=Spectral11, 
                         factors=group.board_disposition.unique(),
                         end=1)

r = figure(plot_width=800, plot_height=300, 
           title="Distribution of CCRB Dispositions",
           x_range=group.allegation.unique(),
           toolbar_location=None,
           tooltips=[("allegation", "@allegation"),
                     ("count", "@rank"),
                     ("CCRB Board","@board_disposition")])

r.vbar(x='allegation', top='rank',
       width=1, source=group,
       line_color="white",fill_color=index_cmap)

r.y_range.start = 0
r.xgrid.grid_line_color = None
r.xaxis.axis_label = "Allegation"
r.xaxis.major_label_orientation = 1.25
r.outline_line_color = None

show(r)

In [14]:
x_tab = pd.crosstab(test_df.board_disposition, test_df.allegation)
data = pd.DataFrame(x_tab.stack(), columns=['rate']).reset_index()
heat_source = ColumnDataSource(data)
    
mapper = LinearColorMapper(palette=Spectral11,
                           low=data.rate.min(),
                           high=data.rate.max())

s = figure(plot_width=800,
           plot_height=500, 
           title="Allegation and CCRB Disposition Co-occurrences",
           x_axis_label="CCRB Disposition",
           y_axis_label="Allegation",
           x_range=data.board_disposition.unique(),
           y_range=data.allegation.unique(),
           toolbar_location=None,
           tools="",
           x_axis_location="below",
           y_axis_location="right")

s.rect(x="board_disposition",
       y="allegation",
       width=1, height=1,
       source=heat_source,
       line_color="white",
       fill_color=transform('rate', mapper))

color_bar = ColorBar(color_mapper=mapper,
                     ticker=BasicTicker(desired_num_ticks=10),
                     label_standoff=7)

s.add_layout(color_bar, "left")
s.xaxis.major_label_orientation = 1.0

# Hover and Tooltips
tooltips = [('CCRB', '@board_disposition'),
            ('Allegation', '@allegation'),
            ('Count', '@rate')]

s.add_tools(HoverTool(tooltips=tooltips,
                      mode='mouse'))

show(s)   

## heatmap of outcomes

In [15]:
x_tab = pd.crosstab(test_df.penalty, test_df.allegation)
data = pd.DataFrame(x_tab.stack(), columns=['rate']).reset_index()
heat_source = ColumnDataSource(data)
    
mapper = LinearColorMapper(palette=Spectral11,
                           low=data.rate.min(),
                           high=data.rate.max())

t = figure(plot_width=800,
           plot_height=500, 
           title="Allegation and Final Penalty Co-occurrences",
           x_axis_label="Final Penalty",
           y_axis_label="Allegation",
           x_range=data.penalty.unique(),
           y_range=data.allegation.unique(),
           toolbar_location=None,
           tools="",
           x_axis_location="below",
           y_axis_location="right")

t.rect(x="penalty",
       y="allegation",
       width=1, height=1,
       source=heat_source,
       line_color="white",
       fill_color=transform('rate', mapper))

color_bar = ColorBar(color_mapper=mapper,
                     ticker=BasicTicker(desired_num_ticks=10),
                     label_standoff=7)

t.add_layout(color_bar, "left")
t.xaxis.major_label_orientation = 1.0

# Hover and Tooltips
tooltips = [('Penalty', '@penalty'),
            ('Allegation', '@allegation'),
            ('Count', '@rate')]

t.add_tools(HoverTool(tooltips=tooltips,
                      mode='mouse'))

show(t)   

### line/bar plot of timeline of complaints

In [16]:
test_df['year'] = test_df.incident_date.dt.year
# test_df.year = test_df.year.astype(str)

/Users/jeffshamp/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [17]:
line_plot = test_df.groupby('year').count()["rank"].reset_index()
line_plot

,year,rank
0,2009,1
1,2011,4
2,2012,7
3,2013,1
4,2014,4


In [18]:
test_df['year'] = test_df.incident_date.dt.year
line_plot = test_df.groupby('year').count()["rank"].reset_index()

e = figure(title = f"Yearly Totals of Complaints",
           x_axis_label = "Year", 
           y_axis_label = "Count", 
           #x_axis_type='datetime', 
           plot_width = 800,
           plot_height = 300,
           toolbar_location = 'above',
           tools='box_zoom, reset')
e.vbar(x='year', top='rank',
       source=line_plot)
show(e)

/Users/jeffshamp/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [19]:
name_title = test_df.groupby("full_name").count()["rank"].reset_index().full_name.item()
name_title

'Daniel Pantaleo'

In [20]:
table = pd.pivot_table(test_df, 
                   index="full_name",columns=["fado_type", "allegation"],
                       aggfunc=np.count_nonzero, margins=True)


In [21]:
table_filter = table.stack(["allegation"])["rank"].reset_index()
table_filter = table_filter[(table_filter.full_name != "All")].drop("All", axis=1)
fados = test_df.fado_type.unique()
table_filter

fado_type,full_name,allegation,Abuse of Authority,Discourtesy,Force
0,Daniel Pantaleo,,NaN,NaN,NaN
1,Daniel Pantaleo,chokehold,NaN,NaN,1.0
2,Daniel Pantaleo,force,NaN,NaN,2.0
3,Daniel Pantaleo,frisk,1.0,NaN,NaN
4,Daniel Pantaleo,hit against object,NaN,NaN,1.0
5,Daniel Pantaleo,nasty words,NaN,1.0,NaN
6,Daniel Pantaleo,other,NaN,NaN,1.0
7,Daniel Pantaleo,refusal - medical treatment,1.0,NaN,NaN
8,Daniel Pantaleo,search - person,1.0,NaN,NaN
9,Daniel Pantaleo,search - vehicle,3.0,NaN,NaN


In [22]:
table = table_filter.melt(id_vars=["full_name", "allegation"])

In [23]:
table

,full_name,allegation,fado_type,value
0,Daniel Pantaleo,,Abuse of Authority,NaN
1,Daniel Pantaleo,chokehold,Abuse of Authority,NaN
2,Daniel Pantaleo,force,Abuse of Authority,NaN
3,Daniel Pantaleo,frisk,Abuse of Authority,1.0
4,Daniel Pantaleo,hit against object,Abuse of Authority,NaN
5,Daniel Pantaleo,nasty words,Abuse of Authority,NaN
6,Daniel Pantaleo,other,Abuse of Authority,NaN
7,Daniel Pantaleo,refusal - medical treatment,Abuse of Authority,1.0
8,Daniel Pantaleo,search - person,Abuse of Authority,1.0
9,Daniel Pantaleo,search - vehicle,Abuse of Authority,3.0
